# Dealing with Large Datasets and Data Prepration for Tuning Model

In [ ]:
from utils import authenticate
credentials, PROJECT_ID = authenticate() 

In [ ]:
REGION = "us-central1"

In [ ]:
import vertexai

In [ ]:
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

In [ ]:
from google.cloud import bigquery

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID,
                            credentials = credentials)

  using [Stack Overflow Data](https://cloud.google.com/blog/topics/public-datasets/google-bigquery-public-datasets-now-include-stack-overflow-q-a) on BigQuery Public Datasets.

In [ ]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [ ]:
query_job = bq_client.query(QUERY_TABLES)

In [ ]:
for row in query_job:
    for value in row.values():
        print(value)

## Data Retrieval

In [ ]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [ ]:
import pandas as pd

In [ ]:
query_job = bq_client.query(INSPECT_QUERY)

In [ ]:
stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
stack_overflow_df.head()

In [ ]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

In [ ]:
query_job = bq_client.query(QUERY_ALL)

In [ ]:
try:
    stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)

**Note:** The data is too large to return, as it is not fitting into memory.

#### Joining Tables and Query Optimization

- When working with (large) data, query optimizing is needed in order to save time and resources.
- Select questions as `input_text` (column 1), answers as `output_text` (column 2).
- Take the questions from `posts_questions` and answers from `posts_answers`.
- Join the questions and their corresponding accepted answers based on their same `unique ID`.
- Making sure the question is about `Python`, and that it `has an answer`. And the date the question was posted is on or after `2020-01-01`
- Limit as 10,000

In [ ]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [ ]:
query_job = bq_client.query(QUERY)

In [ ]:
### this may take some seconds to run
stack_overflow_df = query_job.result()\
                        .to_arrow()\
                        .to_pandas()

stack_overflow_df.head(2)

In [ ]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

In [ ]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' '\
    + stack_overflow_df['input_text']

### Dataset for Tuning



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, evaluation = train_test_split(
    stack_overflow_df,
    ### test_size=0.2 means 20% for evaluation
    ### which then makes train set to be of 80%
    test_size=0.2,
    random_state=42
)

#### Different Datasets and Flow

- Versioning data.

In [ ]:
import datetime

In [ ]:
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")

In [ ]:
cols = ['input_text_instruct','output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [ ]:
training_data_filename = f"tune_data_stack_overflow_\
                            python_qa-{date}.jsonl"

In [ ]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)